This note book is to do experiment on building continuous contracts based on rule: 14 days before each expiration of single contract

In [72]:
import glob
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
%matplotlib inline

In [2]:
# Symbol info frame defines symbols representations in iqfeed and ib, separately.
symbol_info_frame = pd.read_csv(r"E:\project\trade_engine\iqfeed_data_processor\futures_to_import.csv")

In [3]:
# Use glob to find out all the day bar data files in the folder defined
day_bar_data_folder = r"D:\data\iqfeed\future\future_data_day_synthesis"
day_bar_data_path_list = glob.glob(os.path.join(day_bar_data_folder, "*.csv"))

# According to data paths, derived the corresponding iqfeed symbols roots.
# Symbols containing # means it is continuous future, and I do not want to use the continuous future that IB provides
iqfeed_symbol_list = [os.path.basename(path)[:-8] for path in day_bar_data_path_list if "#"not in path]
iqfeed_symbol_root_list = [symbol[:-1] for symbol in iqfeed_symbol_list]
iqfeed_symbol_root_list = list(set(iqfeed_symbol_root_list))

In [4]:
len(iqfeed_symbol_root_list)

22

In [5]:
iqfeed_symbol_root_list

['@M2K',
 'IHO',
 '@MNQ',
 '@KC',
 '@DA',
 '@YC',
 '@MYM',
 '@ES',
 '@OJ',
 '@QG',
 '@QO',
 '@MES',
 'QMGC',
 '@NQ',
 '@QM',
 '@YM',
 '@CT',
 '@SB',
 '@RTY',
 'QGC',
 '@QC',
 '@CC']

In [6]:
frame = pd.read_csv(r"D:\data\iqfeed\future\future_data_day_synthesis\@CCH10_1.csv")

In [7]:
frame = frame.loc[:, ['date', 'open', 'high', 'low', 'close', 'volume']]

In [77]:
for symbol_root in iqfeed_symbol_root_list:

    # According to symbol info frame, figure out the ib symbol root from iqfeed symbol root
    ib_symbol_root = symbol_info_frame[symbol_info_frame["symbol_roots"] == symbol_root].loc[:,"symbol_ib_roots"].iloc[0]

    # Find out all the data path that is under curerent iqfeed symbol root
    current_path_list = [path for path in day_bar_data_path_list if (symbol_root in path and "#" not in path)]

    # combine all of the sub symbols frames into one frame
    combined_frame = pd.DataFrame()

    for path in current_path_list:
        frame = pd.read_csv(path)
        frame = frame.loc[:, ['date', 'open', 'high', 'low', 'close', 'volume']]

        # According to iqfeed day bar data path string, figure out the iqfeed symbol root
        symbol_time_code = os.path.basename(path)[-9:-6]
        ib_symbol = "".join([ib_symbol_root, symbol_time_code])
        frame.loc[:, "symbol"] = ib_symbol

        # Change date time format to python datetime
        datetime_format = '%Y-%m-%d'
        frame['date'] = pd.to_datetime(frame['date'], format=datetime_format)

        # Make sure all data columns are floats
        float_columns = ['open', 'high', 'low', 'close', 'volume']
        for col in float_columns:
            frame.loc[:, col] = frame[col].astype('float')

        # Add a column indicating how many days remained for this future contract
        frame.loc[:, "remain_days"] = frame.index[::-1] 

        # there has to be at least more than 100 days to be included into main contract.
        if len(frame) > 100:
            combined_frame = pd.concat([combined_frame, frame])

    # we will roll contracts 14 days before expiration of each sub future.
    # Here I find out 14 days before expiration of each sub contract
    contracts_roll_info_frame = combined_frame[combined_frame.loc[:,"remain_days"] == 14].loc[:, ["symbol", "date"]]
    contracts_roll_info_frame = contracts_roll_info_frame.sort_values("date")
    contracts_roll_info_frame = contracts_roll_info_frame.reset_index()

    # Remove latest contracts because I do not know which one to use just according to csv file
    do_not_consider_time = datetime.now() - timedelta(days=60)
    contracts_roll_info_frame = contracts_roll_info_frame[contracts_roll_info_frame["date"] < do_not_consider_time]
    
    # Build a continuous future frame
    continuous_future_frame = pd.DataFrame()

    # Variables to be used for contracts calculation
    # Logic is based on link: https://zhuanlan.zhihu.com/p/51880559
    previous_contract_close = 0
    current_contract_open = 0
    accumulative_ratio = 1.0

    # Iterate through contracts roll info frame, and insert data to the continuous contract
    for index, row in contracts_roll_info_frame.iterrows():
        # do no insert data for the first row, since I do not know in which day I should start the first contract
        if index > 0:

            # Get data for the corresponding symbol, according to start and end date
            start_date = contracts_roll_info_frame.loc[index-1, "date"]
            end_date = contracts_roll_info_frame.loc[index,"date"]
            symbol = row.loc["symbol"]
            current_frame = combined_frame[combined_frame["symbol"] == symbol]
            current_frame = current_frame[current_frame["date"] >= start_date]
            current_frame = current_frame[current_frame["date"] < end_date]

            # Here I calculate late a column called multiply ratio. logic based on https://zhuanlan.zhihu.com/p/51880559
            if index == 1:
                current_frame.loc[:, "multiply_ratio"] = 1.0
                accumulative_ratio = 1.0
                previous_contract_close = current_frame.iloc[len(current_frame)-1,:]["close"]
            else:
                current_contract_open = current_frame.iloc[0,:]["open"]
                current_ratio = current_contract_open/previous_contract_close
                accumulative_ratio = accumulative_ratio * current_ratio
                current_frame.loc[:, "multiply_ratio"] = accumulative_ratio
                previous_contract_close = current_frame.iloc[len(current_frame)-1,:]["close"]
                # Multiply the accumulative ratio to the data 
                for column in ['open', 'high', 'low', 'close']:
                    current_frame.loc[:, column] = current_frame.loc[:, column] / accumulative_ratio


            continuous_future_frame = pd.concat([continuous_future_frame, current_frame])

    continuous_future_frame = continuous_future_frame.round({'open': 4, 'high': 4, 'low': 4, 'close': 4, 'multiply_ratio': 6})
    continuous_future_frame.to_csv(ib_symbol_root + ".csv", index=False)